In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from tqdm import tqdm
import time
import bs4 
import requests
import pandas as pd

username = "ikwamkid@outlook.com"
password = "12345678"

driver = webdriver.Chrome(executable_path=r'C:\Users\Admin\Documents\GitHub\tryme_selenium\chromedriver\chromedriver')
driver.get('https://www.dataforthai.com/business/objective/32111')

# กดปุ่ม log in
time.sleep(2)
login = driver.find_element_by_xpath('/html/body/div[2]/div/div[1]/div[1]/div/div[2]/div/div/ul/li[3]/a').click()

# ป้อน username/password
time.sleep(2)
user = driver.find_element_by_xpath('/html/body/div[1]/div/div[4]/div/div/div[2]/form/div[1]/input')
pas = driver.find_element_by_xpath('/html/body/div[1]/div/div[4]/div/div/div[2]/form/div[2]/input')
time.sleep(1)
user.send_keys(username)
time.sleep(1)
pas.send_keys(password)

# กดเข้าสู่ระบบ
time.sleep(1)
btn = driver.find_element_by_id('btn-login').click()

# กดปุ่มเริ่มใช้งาน dataforthai
time.sleep(2)
start = driver.find_element_by_xpath('/html/body/div[1]/div/div[4]/div/div/div/a[1]').click()

# ปิด cookie policy
time.sleep(1)
cookie_button = driver.find_element_by_link_text('ยอมรับ').click()

#Zoom out เพื่อให้ load ทั่วหน้าจอ
driver.execute_script("document.body.style.Transform='scale(0.1)';")
driver.execute_script('document.body.style.TransformOrigin = "0 0";')

## Scraping Process

In [2]:
name =[]
reg_cap = []
province = []
district = []

#เก็บurlผปกจากทั้งหมด4หน้า
for i in tqdm(range(4)):
    url = "https://www.dataforthai.com/business/objective/32111/" + str(i+1)
    driver.get(url)

    #หาชื่อนิติบุคคล
    for n in driver.find_elements_by_xpath('//*[@id="divResult"]/table/tbody/tr/td[2]'):
        name.append(n.text)

    #ทุนจดทะเบียน
    for r in driver.find_elements_by_xpath('//*[@id="divResult"]/table/tbody/tr/td[3]'):
        reg_cap.append(r.text)

    #จังหวัด
    for p in driver.find_elements_by_xpath('//*[@id="divResult"]/table/tbody/tr/td[5]'):
        province.append(p.text)

    #อำเภอ
    for d in driver.find_elements_by_xpath('//*[@id="divResult"]/table/tbody/tr/td[6]'):
        district.append(d.text)

driver.quit() 

100%|██████████| 4/4 [05:07<00:00, 76.91s/it]


In [3]:
table = pd.DataFrame([name,reg_cap,province,district]).transpose()
table.columns = ['name','reg_cap','province','district']
table

,name,reg_cap,province,district
0,168 เคซีพี,"15,000,000",กรุงเทพมหานคร,เขตยานนาวา
1,168 สตูดิโอ แอนด์ ซัพพลาย,"1,000,000",กรุงเทพมหานคร,เขตมีนบุรี
2,2 เอ็มเอ็น,"4,000,000",กรุงเทพมหานคร,เขตบางบอน
3,3 เจ จิวเวลรี่,"3,000,000",กรุงเทพมหานคร,เขตจอมทอง
4,484,"1,000,000",กรุงเทพมหานคร,เขตมีนบุรี
...,...,...,...,...
1427,แฮมิลตันส์,"10,000,000",กรุงเทพมหานคร,เขตบางขุนเทียน
1428,โฮเรีย จิวเวลรี่,"4,000,000",กรุงเทพมหานคร,เขตมีนบุรี
1429,ไฮ-ไท อิมปอร์ต เอ็กซ์ปอร์ต,"1,000,000",กรุงเทพมหานคร,เขตจอมทอง
1430,ไฮท์ คอร์ปอเรชั่น,"40,000,000",กรุงเทพมหานคร,เขตบางรัก


- dump data to json file

In [ ]:
import json
dataforthai = []

In [ ]:
table = table.to_dict('records')
#print (table)

In [ ]:
print (f"page:{len(table)}")

In [ ]:
with open('dataforthai.json', 'w') as f:
    json.dump(table, f)

## Upload data to DataPlatform

In [ ]:
from dsmlibrary.datanode import DataNode

In [ ]:
dir_rawData_id = 87

In [ ]:
datanode = DataNode()

In [ ]:
datanode.upload_file(directory_id=dir_rawData_id, file_path='dataforthai.json')